# Colab formalities

## Connect to gDrive

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

## Download PyTorch

In [0]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision tensorboardX

## Change to ConvCRF directory

In [0]:
import os
os.chdir('/content/drive/My Drive/colab/ConvCRF')

## In case of PIL error

In [0]:
# ! pip uninstall -y pillow
# ! pip install pillow==5.3.0
# import PIL
# PIL.__version__

'5.3.0'

# Train CRF from Deeplab Unary

In [0]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [0]:
! pip install dotmap

In [0]:
import os
import sys
import cv2
import scipy
import timeit
import time
import warnings
import logging as logger
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from dotmap import DotMap
from PIL import Image

import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms

from convcrf.convcrf import GaussCRF, default_conf
from utils.metrics import Metrics, Averages
from demo import do_crf_inference

logger.basicConfig(format='%(asctime)s %(levelname)s %(message)s',
                    level=logger.INFO,
                    stream=sys.stdout)

warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logger.info('Device is {}'.format(device))

## Load Data

In [0]:
from torch.utils.data import DataLoader, WeightedRandomSampler
from utils.pascal_loader_old import PascalDatasetLoader

path = '/content/drive/My Drive/Research/Datasets/VOCdevkit/VOC2012'
traincrf_dataset = PascalDatasetLoader(path, split='train', img_size=513)
val_dataset = PascalDatasetLoader(path, split='val', img_size=513)

num_classes = 21
traincrf_loader = DataLoader(traincrf_dataset, num_workers=4, shuffle=True, batch_size=1)
val_loader = DataLoader(val_dataset, num_workers=4, batch_size=1)

In [0]:
list(range(10))

## Load the image and normalize

In [0]:
import cv2
def load_image_tensor(img_path):

  img = np.zeros((513,513,3));
#   img[:,:,::-1]
  img_temp = cv2.imread(img_path).astype(float)
  img_original = img_temp
  img_temp[:,:,0] = img_temp[:,:,0] - 104.008
  img_temp[:,:,1] = img_temp[:,:,1] - 116.669
  img_temp[:,:,2] = img_temp[:,:,2] - 122.675
  img[:img_temp.shape[0],:img_temp.shape[1],:] = img_temp
  return torch.from_numpy(img[np.newaxis, :].transpose(0,3,1,2)).float()

## Define Deeplab model

In [0]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
import torch
import numpy as np
affine_par = True


def outS(i):
    i = int(i)
    i = (i+1)/2
    i = int(np.ceil((i+1)/2.0))
    i = (i+1)//2
    return i
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes, affine = affine_par)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes, affine = affine_par)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1,  dilation_ = 1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False) # change
        self.bn1 = nn.BatchNorm2d(planes,affine = affine_par)
        
        for i in self.bn1.parameters():
          i.requires_grad = False
        padding = 1
        if dilation_ == 2:
	        padding = 2
        elif dilation_ == 4:
	        padding = 4
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, # change
                               padding=padding, bias=False, dilation = dilation_)
        self.bn2 = nn.BatchNorm2d(planes,affine = affine_par)
        for i in self.bn2.parameters():
            i.requires_grad = False
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4, affine = affine_par)
        for i in self.bn3.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride



    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out

class Classifier_Module(nn.Module):

    def __init__(self,dilation_series,padding_series,NoLabels):
        super(Classifier_Module, self).__init__()
        self.conv2d_list = nn.ModuleList()
        for dilation,padding in zip(dilation_series,padding_series):
    	    self.conv2d_list.append(nn.Conv2d(2048,NoLabels,kernel_size=3,stride=1, padding =padding, dilation = dilation,bias = True))

        for m in self.conv2d_list:
          m.weight.data.normal_(0, 0.01)


    def forward(self, x):
      out = self.conv2d_list[0](x)
      for i in range(len(self.conv2d_list)-1):
        out += self.conv2d_list[i+1](x)
      return out

class ResNet(nn.Module):
    def __init__(self, block, layers,NoLabels):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64,affine = affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True) # change
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1, dilation__ = 2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation__ = 4)
        self.layer5 = self._make_pred_layer(Classifier_Module, [6,12,18,24],[6,12,18,24],NoLabels)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, 0.01)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        #        for i in m.parameters():
        #            i.requires_grad = False

    def _make_layer(self, block, planes, blocks, stride=1,dilation__ = 1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion or dilation__ == 2 or dilation__ == 4:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion,affine = affine_par),
            )
        for i in downsample._modules['1'].parameters():
            i.requires_grad = False
        layers = []
        layers.append(block(self.inplanes, planes, stride,dilation_=dilation__, downsample = downsample ))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes,dilation_=dilation__))

        return nn.Sequential(*layers)
        
    def _make_pred_layer(self,block, dilation_series, padding_series,NoLabels):
	    return block(dilation_series,padding_series,NoLabels)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)

        return x

class MS_Deeplab(nn.Module):
    def __init__(self,block,NoLabels):
    	super(MS_Deeplab,self).__init__()
    	self.Scale = ResNet(block,[3, 4, 23, 3],NoLabels)   #changed to fix #4 

    def forward(self,x):
        input_size = x.size()[2]
        self.interp1 = nn.UpsamplingBilinear2d(size=(int(input_size*0.75)+1, int(input_size*0.75)+1))
        self.interp2 = nn.UpsamplingBilinear2d(size=(int(input_size*0.5)+1, int(input_size*0.5)+1))
        self.interp3 = nn.UpsamplingBilinear2d(size=(outS(input_size), outS(input_size)))
        out = []
        x2 = self.interp1(x)
        x3 = self.interp2(x)
        out.append(self.Scale(x))	# for original scale
        out.append(self.interp3(self.Scale(x2)))	# for 0.75x scale
        out.append(self.Scale(x3))	# for 0.5x scale


        x2Out_interp = out[1]
        x3Out_interp = self.interp3(out[2])
        temp1 = torch.max(out[0],x2Out_interp)
        out.append(torch.max(temp1,x3Out_interp))
        return out

def Res_Deeplab(NoLabels=21):
    model = MS_Deeplab(Bottleneck,NoLabels)
    return model

## Load deeplab weights


In [0]:
model_path = '/content/drive/My Drive/Research/Models/Pretrained/MS_DeepLab_resnet_trained_VOC.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
deeplab_model = Res_Deeplab()
deeplab_model.load_state_dict(torch.load(model_path))

deeplab_model = deeplab_model.to(device)
deeplab_model.eval()

## Get the Unary from Deeplab output

In [0]:
def get_unary(outputs):
  interp = nn.UpsamplingBilinear2d(size=(513,513))
  output = interp(outputs[3]).cpu().data[0].numpy()
  output = output.transpose(1,2,0)
  output = np.argmax(output,axis = 2)
  return output
#   return output[None, :, :, :]

## Define the CRF model

In [0]:
config = default_conf
crf_model = GaussCRF(conf=config, shape=(500, 500), nclasses=num_classes)
# model.load_state_dict(saved_state['model_state'])
crf_model = crf_model.to(device)

## Define the loss function and optimizer

In [0]:
import torch.optim as optim

criterion= nn.CrossEntropyLoss()
optimizer = optim.Adam(crf_model.parameters(), lr=0.00005)

## Train the network

In [0]:
args = DotMap()
args.pyinn = False
args.nospeed = False
args.output = None

running_metrics = Metrics(num_classes)
train_loss_avg = Averages()
val_loss_avg = Averages()
time_avg = Averages()

# best_iou = saved_state['best_iou']

# logger.info('Starting from iou: {}'.format(best_iou))

num_epochs = 10

In [0]:
images, labels = next(iter(traincrf_loader))
img = images.to(device)
outputs = deeplab_model(img)
o = get_unary(outputs)
np.min(o)

0

In [0]:
from tensorboardX import SummaryWriter
writer = SummaryWriter

In [0]:
for epoch in range(1): 
    running_loss = 0.0
    actual_epoch = epoch + 1
#     scheduler.step()
    crf_model.train()

    for i, (images, labels) in enumerate(traincrf_loader):
        iteration = i + 1

        start_ts = time.time()

        images = images.to(device)

        optimizer.zero_grad()

#         labels = labels[0]
#         unary = augment_label(labels, num_classes=num_classes)
        unary = batch_augment_label(labels)
#         unary = unary.transpose(2, 0, 1).reshape([1, num_classes, unary.shape[0], unary.shape[1]])

        unary = torch.from_numpy(unary).float().to(device)
        labels = labels.to(device)
        
        outputs = model(unary=unary, img=images)
        
        outputs = outputs.transpose(1,2).transpose(2,3).contiguous().view(-1, 21)
        labels = labels.view(-1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_loss_avg.update(loss.item())
        time_avg.update(time.time() - start_ts)

        if iteration % 100 == 0:
            avg_loss = train_loss_avg.avg
            print('[{:d}, {:d}] Average loss: {:.4f} Average Time: {:.4f} Learning rate: {}'
                  .format(actual_epoch, iteration, avg_loss, time_avg.avg, optimizer.param_groups[0]['lr']))
            
            train_loss_avg.reset()
            time_avg.reset()
            
    
    with torch.no_grad():
        model.eval()
        val_len = len(val_loader)
        running_val_loss = 0.0
        for i_val, (images_val, labels_val) in enumerate(val_loader):
            iter_val = i_val + 1
#             labels_val = labels_val[0] # remove batch dimension
#             unary = augment_label(labels_val, num_classes=num_classes)
            unary = batch_augment_label(labels_val)

#             unary = unary.transpose(2, 0, 1).reshape([1, num_classes, unary.shape[0], unary.shape[1]])
            unary = torch.from_numpy(unary).float().to(device)

            images_val = images_val.to(device)
            labels_val = labels_val.to(device)

            predictions = model(unary=unary, img=images_val)
            pred = predictions.transpose(1,2).transpose(2,3).contiguous().view(-1, 21)

            labels = labels_val.view(-1)
            val_loss = criterion(pred, labels)
            

            preds_np = predictions.data.max(1)[1].cpu().numpy()
            labels_np = labels_val.data.cpu().numpy()

            running_metrics.update(labels_np, preds_np)
            val_loss_avg.update(val_loss.item())
            
            running_val_loss += val_loss.item()

            if iter_val % 50 == 0:
                print("{}/{} Loss: {}: ".format(iter_val, val_len, val_loss_avg.avg))
                val_loss_avg.reset()

    score, class_iou = running_metrics.get_scores()
    
    print('\n Avg train loss: {:.5f} vs Avg val loss: {:.5f}'
          .format(running_loss/len(traincrf_loader), running_val_loss/len(val_loader)))

    print('\nEpoch: {} Validation Summary'.format(actual_epoch))
    for k, v in score.items():
        print(k, v)
    #   writer.add_scalar('val_metrics/{}'.format(k), v, i+1)

    running_metrics.reset()

    if score["Mean IoU : \t"] >= best_iou:
        best_iou = score["Mean IoU : \t"]
        logger.info('Found new best_iou: {}'.format(best_iou))
        state = {
                "epoch": actual_epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "best_iou": best_iou,
                }
        logger.info(save_path)
        torch.save(state, save_path)